In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load the datasetata = "usedcars.csv"
data = 'usedcars.csv'
df = pd.read_csv(data)
df['price'] = df['price']/1000
df['mileage'] = df['mileage']/1000

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20063 entries, 0 to 20062
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         20063 non-null  float64
 1   trim          20063 non-null  object 
 2   isOneOwner    20063 non-null  object 
 3   mileage       20063 non-null  float64
 4   year          20063 non-null  int64  
 5   color         20063 non-null  object 
 6   displacement  20063 non-null  float64
 7   fuel          20063 non-null  object 
 8   region        20063 non-null  object 
 9   soundSystem   20063 non-null  object 
 10  wheelType     20063 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 1.7+ MB


In [4]:
df.head()

,price,trim,isOneOwner,mileage,year,color,displacement,fuel,region,soundSystem,wheelType
0,2.988,320,f,193.296,1995,Black,3.2,Gasoline,SoA,unsp,Alloy
1,6.595,320,f,129.948,1995,other,3.2,Gasoline,Mid,Premium,Alloy
2,7.993,320,f,140.428,1997,White,3.2,Gasoline,Mid,Bose,Alloy
3,5.995,420,f,113.622,1999,Silver,4.2,Gasoline,Mid,unsp,Alloy
4,3.000,420,f,167.673,1999,Silver,4.2,Gasoline,SoA,unsp,Alloy


In [5]:
df.describe()

,price,mileage,year,displacement
count,20063.000000,20063.000000,20063.000000,20063.000000
mean,30.747244,73.113479,2007.010018,5.058964
std,18.291085,43.075537,4.128162,0.628131
min,0.599000,0.008000,1994.000000,3.000000
25%,13.495000,39.888500,2004.000000,4.600000
50%,29.454000,67.187000,2007.000000,5.500000
75%,43.995000,98.213000,2010.000000,5.500000
max,79.999000,488.525000,2014.000000,6.300000


In [6]:
features = ['trim', 'isOneOwner', 'year', 'color', 'displacement', 'fuel', 'region', 'soundSystem', 'wheelType']
for feature in features:
  print(feature)
  print(df[feature].value_counts())
  print()

trim
550       11825
430        2787
500        2661
63 AMG      599
600         572
55 AMG      356
350         352
320         274
420         239
400         220
65 AMG      178
Name: trim, dtype: int64

isOneOwner
f    16594
t     3469
Name: isOneOwner, dtype: int64

year
2007    3607
2008    2352
2012    1872
2011    1625
2010    1624
2013    1169
2006    1056
2009     996
2003     911
2002     903
2004     847
2000     823
2001     789
2005     685
1995     224
1999     170
1997     142
1996     106
1998      93
1994      64
2014       5
Name: year, dtype: int64

color
Black     8194
Silver    4353
White     2641
Gray      2168
other      961
Blue       914
unsp       832
Name: color, dtype: int64

displacement
5.5    9561
4.6    2794
4.3    2787
5.0    2661
6.3     494
5.4     356
3.2     274
4.2     239
3.5     227
6.0     213
3.0     204
3.7     141
5.8     112
Name: displacement, dtype: int64

fuel
Gasoline    19632
Hybrid        220
Diesel        211
Name: fuel, dtype: int64

In [7]:
df.replace("unsp", float("NaN"), inplace=True) 
df.isna().sum()

price              0
trim               0
isOneOwner         0
mileage            0
year               0
color            832
displacement       0
fuel               0
region             0
soundSystem     9100
wheelType       8404
dtype: int64

In [8]:
# columns_with_missing_values = ["color", "soundSystem", "wheelType"]
columns_with_missing_values = ["color"]

# Convert categorical columns with missing values to numerical columns using ordinal encoding
ordinal_encoder = OrdinalEncoder()
df[columns_with_missing_values] = ordinal_encoder.fit_transform(df[columns_with_missing_values])

In [9]:
df

,price,trim,isOneOwner,mileage,year,color,displacement,fuel,region,soundSystem,wheelType
0,2.988,320,f,193.296,1995,0.0,3.2,Gasoline,SoA,NaN,Alloy
1,6.595,320,f,129.948,1995,5.0,3.2,Gasoline,Mid,Premium,Alloy
2,7.993,320,f,140.428,1997,4.0,3.2,Gasoline,Mid,Bose,Alloy
3,5.995,420,f,113.622,1999,3.0,4.2,Gasoline,Mid,NaN,Alloy
4,3.000,420,f,167.673,1999,3.0,4.2,Gasoline,SoA,NaN,Alloy
...,...,...,...,...,...,...,...,...,...,...,...
20058,67.950,550,t,17.181,2013,0.0,4.6,Gasoline,WSC,Harman Kardon,Alloy
20059,31.995,400,t,53.885,2010,0.0,3.5,Hybrid,SoA,NaN,NaN
20060,34.995,400,f,47.484,2010,0.0,3.5,Hybrid,WSC,NaN,Alloy
20061,38.991,400,f,42.972,2010,4.0,3.5,Hybrid,SoA,NaN,Alloy


In [10]:
# Define the imputer
cols_to_impute = ['color']
imputer = IterativeImputer(max_iter=10)
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])

In [11]:
# Convert the numerical columns back to categorical columns
df[cols_to_impute] = ordinal_encoder.inverse_transform(df[cols_to_impute])

In [12]:
df = df.drop(['soundSystem','wheelType'], axis=1)

In [13]:
df.head()

,price,trim,isOneOwner,mileage,year,color,displacement,fuel,region
0,2.988,320,f,193.296,1995,Black,3.2,Gasoline,SoA
1,6.595,320,f,129.948,1995,other,3.2,Gasoline,Mid
2,7.993,320,f,140.428,1997,White,3.2,Gasoline,Mid
3,5.995,420,f,113.622,1999,Silver,4.2,Gasoline,Mid
4,3.000,420,f,167.673,1999,Silver,4.2,Gasoline,SoA


In [14]:
df.isna().sum()

price           0
trim            0
isOneOwner      0
mileage         0
year            0
color           0
displacement    0
fuel            0
region          0
dtype: int64

In [15]:
# find duplicate rows based on all columns
duplicate_rows = df[df.duplicated()]

# print the duplicate rows
print("Duplicate Rows except first occurrence based on all columns are:")
print(duplicate_rows)

Duplicate Rows except first occurrence based on all columns are:
        price trim isOneOwner  mileage  year   color  displacement      fuel  \
1508    4.000  430          f   57.453  2004   Black           4.3  Gasoline   
7330    4.500  430          f   72.141  2005   Black           4.3  Gasoline   
13137   2.800  430          f   59.690  2002   Black           4.3  Gasoline   
15708  16.988  600          f   48.501  2002   Black           5.8  Gasoline   
19596   9.999  430          f   99.706  2003  Silver           4.3  Gasoline   

      region  
1508     Mid  
7330     Mid  
13137    Mid  
15708    WNC  
19596    SoA  


In [17]:
features = ['trim', 'isOneOwner', 'year', 'color', 'displacement', 'fuel', 'region']
for feature in features:
  print(feature)
  print(df[feature].value_counts())
  print()

trim
550       11825
430        2787
500        2661
63 AMG      599
600         572
55 AMG      356
350         352
320         274
420         239
400         220
65 AMG      178
Name: trim, dtype: int64

isOneOwner
f    16594
t     3469
Name: isOneOwner, dtype: int64

year
2007    3607
2008    2352
2012    1872
2011    1625
2010    1624
2013    1169
2006    1056
2009     996
2003     911
2002     903
2004     847
2000     823
2001     789
2005     685
1995     224
1999     170
1997     142
1996     106
1998      93
1994      64
2014       5
Name: year, dtype: int64

color
Black     8194
Silver    4353
White     2641
Gray      2168
Blue      1746
other      961
Name: color, dtype: int64

displacement
5.5    9561
4.6    2794
4.3    2787
5.0    2661
6.3     494
5.4     356
3.2     274
4.2     239
3.5     227
6.0     213
3.0     204
3.7     141
5.8     112
Name: displacement, dtype: int64

fuel
Gasoline    19632
Hybrid        220
Diesel        211
Name: fuel, dtype: int64

region
SoA   

In [18]:
df.select_dtypes(include='object').nunique()

trim          11
isOneOwner     2
color          6
fuel           3
region         9
dtype: int64

In [19]:
df.select_dtypes(include='int').nunique()

year    21
dtype: int64

In [20]:
df.select_dtypes(include='float').nunique()

price            4992
mileage         18231
displacement       13
dtype: int64

In [16]:
df.to_csv('cleaneddata.csv', index=False)